In [1]:
import pandas as pd
import requests
import datetime

icaos = ["EDDB", "EDDH"]

def get_arrivals(icaos):

  flights_df_list = []

  for icao in icaos:

    tomorrow = datetime.date.today() + datetime.timedelta(days=1)
    times = [f"{tomorrow}T00:00/{tomorrow}T11:59",f"{tomorrow}T12:00/{tomorrow}T23:59"]

    for time in times:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{time}"

      querystring = {"withLeg":"true","direction":"Both","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

      headers = {
        "X-RapidAPI-Key": "YOUR_API_KEY",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
      }

      response = requests.get(url, headers=headers, params=querystring)

      arrivals_df = pd.json_normalize(response.json()["arrivals"])
      arrivals_df_small = arrivals_df[["number", "status", "departure.airport.name", "departure.actualTimeLocal", "arrival.scheduledTimeLocal"]].copy()

      arrivals_df_small = arrivals_df_small.rename(columns={"number": "flight_number",
                                                            "status": "flight_status",
                                                            "departure.airport.name": "departure_city_name",
                                                            "departure.actualTimeLocal": "departure_time_local",
                                                            "arrival.scheduledTimeLocal": "arrival_time_local"})

      flights_df_list.append(arrivals_df_small)

  return pd.concat(flights_df_list, ignore_index=True)

In [2]:
get_arrivals(icaos)

,flight_number,flight_status,departure_city_name,departure_time_local,arrival_time_local
0,XQ 966,Expected,İzmir,2023-07-15 03:40+03:00,2023-07-15 05:45+02:00
1,FI 518,Expected,Reykjavik,2023-07-15 00:45+00:00,2023-07-15 06:15+02:00
2,IV 680,Expected,Prishtina,NaN,2023-07-15 06:50+02:00
3,A3 3219,Expected,Riga,2023-07-15 07:05+03:00,2023-07-15 07:40+02:00
4,BT 211,Expected,Riga,2023-07-15 07:05+03:00,2023-07-15 07:40+02:00
...,...,...,...,...,...
464,DE 1441,Expected,Fuerteventura Island,2023-07-15 16:45+01:00,2023-07-15 22:20+02:00
465,DE 155,Expected,Antalya,NaN,2023-07-15 22:40+02:00
466,DI 6003,Unknown,Hurghada,NaN,2023-07-15 22:45+02:00
467,DI 6037,Expected,Málaga,2023-07-15 19:20+02:00,2023-07-15 22:35+02:00
